- Author: Peter Riesebos
- Purpose: Script used to look up significant eQTLs from one file (TopEffects) in another (AllEffects) to check for overlap and flipped alleles.
- Input: mbQTL format summary stats files (TopEffets and AllEffects files) from different datasets.
- Output: summary stats of found overlap and flipped alleles

## Imports

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing
import dask.dataframe as dd  # Ensure Dask is imported
import logging
from dask.dataframe import read_csv
import os
import glob

GTEx mbqtl output path:
- /groups/umcg-fg/tmp04/projects/gut-bulk/ongoing/2024-02-07-GutPublicRNASeq/datasets/GTEx/mbQTL_output

Werna mbqtl output path:
- /groups/umcg-fg/tmp04/projects/gut-bulk/ongoing/2024-02-07-GutPublicRNASeq/datasets/Werna/qtl/eqtl/output

Pub RNA mbqtl output path:
- /groups/umcg-fg/tmp04/projects/gut-bulk/ongoing/2024-02-07-GutPublicRNASeq/datasets/pub_rna/final_files_pub_rna/pub_rna_100perm_30_cutoff_maf_0.05

In [28]:
# Configure logging
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    handlers=[logging.StreamHandler()])

# Function to create a unique ID for comparison
def create_unique_id(gene, gene_chr, snp_pos, snp_alleles, flip=False):
    if flip:
        alleles = snp_alleles.split('/')
        flipped_alleles = f"{alleles[1]}/{alleles[0]}"
        return f"{gene}:{gene_chr}:{snp_pos}:{flipped_alleles}"
    else:
        return f"{gene}:{gene_chr}:{snp_pos}:{snp_alleles}"

# Function to compare files
def compare_files(chromosome_file, combined_df, output_dir):
    logging.info(f"Processing file: {chromosome_file}")
    
    # Use memory-mapped files for chromosome data using Dask
    chrom_df = dd.read_csv(chromosome_file, 
                           dtype={'Gene': str, 'GeneChr': str, 'SNPPos': int, 'SNPAlleles': str}, 
                           sep='\t', blocksize='64MB', assume_missing=True)
    
    # Strip the 'Gene' column
    chrom_df['Gene'] = chrom_df['Gene'].str.replace(r'\.\d+$', '', regex=True)

    # Create unique combination of relevant columns for the chromosome file
    chrom_df['UniqueID'] = chrom_df.apply(lambda row: create_unique_id(
        row['Gene'], row['GeneChr'], row['SNPPos'], row['SNPAlleles']), axis=1, meta=('x', 'str'))

    # Add a new column 'Flipped' and set to False by default
    chrom_df['Flipped'] = False

    # Create indexes on `UniqueID` and `UniqueID_Flipped` columns for the combined DataFrame
    combined_df.set_index('UniqueID', inplace=True)
    combined_df_flipped = combined_df.set_index('UniqueID_Flipped')

    total_exact_matches = 0
    total_flipped_matches = 0

    # Iterate through the chromosome file in chunks
    for chunk in chrom_df.partitions:  # Iterate over Dask partitions
        chunk_df = chunk.compute()  # Convert partition to pandas DataFrame

        # Log the size of the current chunk
        logging.info(f"Processing chunk of size: {len(chunk_df)}")

        # Vectorized matching for exact and flipped matches
        exact_match = chunk_df[chunk_df['UniqueID'].isin(combined_df.index)]
        flipped_match = chunk_df[chunk_df['UniqueID'].isin(combined_df_flipped.index)]

        # Update match counts
        total_exact_matches += len(exact_match)
        total_flipped_matches += len(flipped_match)

        # Mark flipped matches
        if not flipped_match.empty:
            flipped_match['Flipped'] = True

        # Combine exact and flipped matches
        matching_rows = pd.concat([exact_match, flipped_match])

        # Save only matching rows to the output directory
        if not matching_rows.empty:
            output_file = os.path.join(output_dir, os.path.basename(chromosome_file).replace('.gz', ''))
            matching_rows.to_csv(output_file, sep='\t', index=False, mode='a', header=not os.path.exists(output_file))
    
    logging.info(f"File processed: {chromosome_file}, Total exact matches: {total_exact_matches}, Total flipped matches: {total_flipped_matches}")


# Function to process chromosome files in parallel
def process_chromosome_files(chromosome_dir, combined_file, output_dir):
    logging.info(f"Starting to process combined file: {combined_file}")
    
    # Read in the combined summary statistics file
    combined_df = pd.read_csv(combined_file, sep='\t')

    # Strip the 'Gene' column in the combined dataset
    combined_df['Gene'] = combined_df['Gene'].str.replace(r'\.\d+$', '', regex=True)

    # Create unique IDs for the combined dataset
    combined_df['UniqueID'] = combined_df.apply(lambda row: create_unique_id(
        row['Gene'], row['GeneChr'], row['SNPPos'], row['SNPAlleles']), axis=1)
    
    combined_df['UniqueID_Flipped'] = combined_df.apply(lambda row: create_unique_id(
        row['Gene'], row['GeneChr'], row['SNPPos'], row['SNPAlleles'], flip=True), axis=1)

    logging.info("Finished processing the combined file and generating unique IDs.")
    
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Use multiprocessing to process chromosome files in parallel
    pool = multiprocessing.Pool()
    chromosome_files = [f for f in os.listdir(chromosome_dir) if f.endswith('AllEffects.txt.gz')]
    
    if not chromosome_files:
        logging.warning(f"No files to process in directory: {chromosome_dir}")
        return
    
    args_list = [(os.path.join(chromosome_dir, chromosome_file), combined_df, output_dir) 
                 for chromosome_file in chromosome_files]
    
    logging.info(f"Processing {len(chromosome_files)} chromosome files in parallel.")
    pool.starmap(compare_files, args_list)
    pool.close()
    pool.join()

    logging.info("Finished processing all chromosome files.")

# Example usage:
chromosome_dir = '/groups/umcg-fg/tmp04/projects/gut-bulk/ongoing/2024-02-07-GutPublicRNASeq/datasets/Werna/qtl/eqtl/output'
combined_file = '/groups/umcg-fg/tmp04/projects/gut-bulk/ongoing/2024-02-07-GutPublicRNASeq/datasets/pub_rna/final_files_pub_rna/pub_rna_100perm_30_cutoff_maf_0.05/merged_topEffects_qvalue.txt'
output_dir = '/groups/umcg-fg/tmp04/projects/gut-bulk/ongoing/2024-02-07-GutPublicRNASeq/datasets/intersect/'

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Process all chromosome files
process_chromosome_files(chromosome_dir, combined_file, output_dir)


2024-09-19 14:55:49,396 - INFO - Starting to process combined file: /groups/umcg-fg/tmp04/projects/gut-bulk/ongoing/2024-02-07-GutPublicRNASeq/datasets/pub_rna/final_files_pub_rna/pub_rna_100perm_30_cutoff_maf_0.05/merged_topEffects_qvalue.txt


2024-09-19 14:55:50,458 - INFO - Finished processing the combined file and generating unique IDs.
2024-09-19 14:55:51,246 - INFO - Processing 22 chromosome files in parallel.
2024-09-19 14:55:51,525 - INFO - Processing file: /groups/umcg-fg/tmp04/projects/gut-bulk/ongoing/2024-02-07-GutPublicRNASeq/datasets/Werna/qtl/eqtl/output/__19-AllEffects.txt.gz
/groups/umcg-fg/tmp04/projects/gut-bulk/ongoing/users/umcg-priesebos/miniforge3/lib/python3.12/site-packages/dask/dataframe/io/csv.py:555: UserWarning: Warning gzip compression does not support breaking apart files
Please ensure that each individual file can fit in memory and
use the keyword ``blocksize=None to remove this message``
Setting ``blocksize=None``
  warn(
2024-09-19 14:55:51,650 - INFO - Processing file: /groups/umcg-fg/tmp04/projects/gut-bulk/ongoing/2024-02-07-GutPublicRNASeq/datasets/Werna/qtl/eqtl/output/__16-AllEffects.txt.gz
/groups/umcg-fg/tmp04/projects/gut-bulk/ongoing/users/umcg-priesebos/miniforge3/lib/python3.12/si

In [19]:
chr1_gtex = pd.read_csv("/groups/umcg-fg/tmp04/projects/gut-bulk/ongoing/2024-02-07-GutPublicRNASeq/datasets/gtex_colon_small_chr1-AllEffects.txt", sep='\t')

In [23]:
def merge_txt_files_to_dataframe(directory_path):
    file_pattern = os.path.join(directory_path, "gtex_colon_small*.txt")
    txt_files = glob.glob(file_pattern)
    
    if not txt_files:
        raise FileNotFoundError(f"No .txt files found in the directory: {directory_path}")
    
    dataframes = []
    
    for file in txt_files:
        try:
            df = pd.read_csv(file, sep='\t')  # Assuming the files are tab-separated
            dataframes.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")
    
    if dataframes:
        merged_df = pd.concat(dataframes, ignore_index=True)
    else:
        raise ValueError("No data could be loaded from the .txt files.")
    
    return merged_df

directory_path = '/groups/umcg-fg/tmp04/projects/gut-bulk/ongoing/2024-02-07-GutPublicRNASeq/datasets/intersect/'
merged_df = merge_txt_files_to_dataframe(directory_path)

# To check the result
merged_df

,Gene,GeneChr,GenePos,GeneStrand,GeneSymbol,SNP,SNPChr,SNPPos,SNPAlleles,SNPEffectAllele,...,MetaPZ,MetaBeta,MetaSE,MetaI2,NrDatasets,DatasetCorrelationCoefficients(GTEx_Colon_Transverse;GTEx_Small_Intestine),DatasetZScores(GTEx_Colon_Transverse;GTEx_Small_Intestine),DatasetSampleSizes(GTEx_Colon_Transverse;GTEx_Small_Intestine),UniqueID,Flipped
0,ENSG00000273321,18,24402153.0,+,ENSG00000273321.1,rs1245472469,18.0,23468293,CT/C,C,...,1.013194,0.105406,0.104034,0.000000,2.0,0.05651;-0.001044,1.073812;-0.006646,363;43,ENSG00000273321:18:23468293:CT/C,False
1,ENSG00000266145,18,14222008.0,+,ENSG00000266145.1,rs4797795,18.0,13676763,G/A,A,...,0.493386,0.034656,0.070242,0.000000,2.0,0.008725;0.161511,0.165671;1.034702,363;43,ENSG00000266145:18:13676763:G/A,False
2,ENSG00000264705,18,69212277.0,+,ENSG00000264705.1,rs11290357,18.0,68833383,GT/G,G,...,0.484151,0.034518,0.071296,0.558832,2.0,-0.001695;0.244654,-0.032174;1.581164,363;43,ENSG00000264705:18:68833383:GT/G,False
3,ENSG00000278075,18,79900555.0,-,ENSG00000278075.1,rs1660159,18.0,79491254,C/G,G,...,-0.636560,-0.046045,0.072335,0.000000,2.0,-0.022435;-0.112487,-0.426028;-0.718176,363;43,ENSG00000278075:18:79491254:C/G,False
4,ENSG00000270469,18,14053951.0,+,ENSG00000270469.1,rs570752551,18.0,13766389,CAAAAA/C,C,...,0.041790,0.003218,0.077002,0.762097,2.0,0.03721;-0.295673,0.706739;-1.925011,363;43,ENSG00000270469:18:13766389:CAAAAA/C,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4329,ENSG00000225193,15,90206572.0,+,ENSG00000225193.5,rs368554712,15.0,89858491,GAA/G,G,...,-0.121197,-0.008749,0.072189,0.000000,2.0,-0.003582;-0.027457,-0.06802;-0.174776,363;43,ENSG00000225193:15:89858491:GAA/G,False
4330,ENSG00000274294,15,99416584.0,+,ENSG00000274294.1,rs1452112097,15.0,98960292,C/CTT,CTT,...,0.676671,0.047473,0.070157,0.736652,2.0,0.000298;0.315778,0.005664;2.062791,363;43,ENSG00000274294:15:98960292:C/CTT,False
4331,ENSG00000260144,15,72605183.0,+,ENSG00000260144.1,rs1878940,15.0,73304482,T/C,C,...,-0.056095,-0.004190,0.074700,0.000000,2.0,0.000201;-0.028817,0.003811;-0.183437,363;43,ENSG00000260144:15:73304482:T/C,False
4332,ENSG00000140350,15,68778535.0,-,ENSG00000140350.15,rs6494726,15.0,68248689,C/T,T,...,-1.051404,-0.074236,0.070607,0.000000,2.0,-0.068333;0.085223,-1.298949;0.543366,363;43,ENSG00000140350:15:68248689:C/T,False


In [29]:
def merge_txt_files_to_dataframe(directory_path):
    file_pattern = os.path.join(directory_path, "__*AllEffects.txt")
    txt_files = glob.glob(file_pattern)
    
    if not txt_files:
        raise FileNotFoundError(f"No .txt files found in the directory: {directory_path}")
    
    dataframes = []
    
    for file in txt_files:
        try:
            df = pd.read_csv(file, sep='\t')  # Assuming the files are tab-separated
            dataframes.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")
    
    if dataframes:
        merged_df_pub_werna = pd.concat(dataframes, ignore_index=True)
    else:
        raise ValueError("No data could be loaded from the .txt files.")
    
    return merged_df_pub_werna

directory_path = '/groups/umcg-fg/tmp04/projects/gut-bulk/ongoing/2024-02-07-GutPublicRNASeq/datasets/intersect/'
merged_df_pub_werna = merge_txt_files_to_dataframe(directory_path)

# To check the result
merged_df_pub_werna

,Gene,GeneChr,GenePos,GeneStrand,GeneSymbol,SNP,SNPChr,SNPPos,SNPAlleles,SNPEffectAllele,...,MetaPZ,MetaBeta,MetaSE,MetaI2,NrDatasets,DatasetCorrelationCoefficients(1000IBD),DatasetZScores(1000IBD),DatasetSampleSizes(1000IBD),UniqueID,Flipped
0,ENSG00000266513,18,3770017.0,-,ENSG00000266513.1,rs10502306,18.0,2780461,A/G,G,...,-1.203835,-0.139737,0.116076,0.0,1.0,-0.083028,-1.203835,212.0,ENSG00000266513:18:2780461:A/G,False
1,ENSG00000267013,18,55105904.0,-,ENSG00000267013.6,rs12607966,18.0,54140910,C/T,T,...,-1.082740,-0.114740,0.105972,0.0,1.0,-0.074701,-1.082740,212.0,ENSG00000267013:18:54140910:C/T,False
2,ENSG00000266145,18,14222008.0,+,ENSG00000266145.1,rs4797795,18.0,13676763,G/A,A,...,1.059594,0.102839,0.097055,0.0,1.0,0.073108,1.059594,212.0,ENSG00000266145:18:13676763:G/A,False
3,ENSG00000166396,18,63752935.0,+,ENSG00000166396.13,rs557825625,18.0,63291726,C/CT,CT,...,-0.176306,-0.017185,0.097473,0.0,1.0,-0.012180,-0.176306,212.0,ENSG00000166396:18:63291726:C/CT,False
4,ENSG00000265142,18,21825487.0,-,ENSG00000265142.9,rs72199360,18.0,22202675,AGT/A,A,...,0.181634,0.018824,0.103640,0.0,1.0,0.012548,0.181634,212.0,ENSG00000265142:18:22202675:AGT/A,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44858,ENSG00000164967,9,34610486.0,-,ENSG00000164967.10,rs1491332777,9.0,33873174,T/TA,TA,...,-0.071648,-0.009539,0.133139,0.0,1.0,-0.004950,-0.071648,212.0,ENSG00000164967:9:33873174:T/TA,True
44859,ENSG00000227383,9,34191555.0,+,ENSG00000227383.1,rs1377628055,9.0,34406803,C/CT,CT,...,-0.466859,-0.055195,0.118227,0.0,1.0,-0.032246,-0.466859,212.0,ENSG00000227383:9:34406803:C/CT,True
44860,ENSG00000233668,9,34223984.0,-,ENSG00000233668.1,rs71506164,9.0,34080190,T/TA,TA,...,0.585042,0.075611,0.129239,0.0,1.0,0.040403,0.585042,212.0,ENSG00000233668:9:34080190:T/TA,True
44861,ENSG00000252943,9,19073843.0,+,ENSG00000252943.1,rs1427784246,9.0,19071573,C/CT,CT,...,-0.812713,-0.106144,0.130604,0.0,1.0,-0.056105,-0.812713,212.0,ENSG00000252943:9:19071573:C/CT,True


In [26]:
# protein coding genes gencode v44
genes = pd.read_csv("/groups/umcg-fg/tmp04/projects/gut-bulk/ongoing/2024-02-07-GutPublicRNASeq/datasets/pub_rna/final_files_pub_rna/protein_coding_genes_list.txt", names=["Gene"])
new_genes = genes.copy()
new_genes['Gene'] = new_genes['Gene'].str.replace(r'\.\d+$', '', regex=True)
new_genes.Gene.head(1)

0    ENSG00000186092
Name: Gene, dtype: object

In [27]:
intersect_gtex = set(new_genes["Gene"]) & set(merged_df["Gene"])
print(len(intersect_gtex))

303


In [24]:
merged_df.Flipped.value_counts()

Flipped
False    4276
True       58
Name: count, dtype: int64

In [30]:
merged_df_pub_werna.Flipped.value_counts()

Flipped
False    44151
True       712
Name: count, dtype: int64